In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
import numpy as np
import os
import random

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.contrib import layers, rnn, seq2seq
from tqdm import tqdm

In [3]:
import fasttext as fs

In [4]:
from data_generator import NLUDataGenerator
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple
from tensorflow.contrib.legacy_seq2seq import rnn_decoder

In [5]:
PAD = 0

TimeMajor = False
batch_size = 32
encoder_max_time = 64

DataGenDev = NLUDataGenerator('../data/dev/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', None, seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)

DataGen = NLUDataGenerator('../data/train/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', DataGenDev.vocab, seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)

vocab_size = len(DataGen.vocab)
input_embedding_size = 100
# encoder_hidden_units = 128
# decoder_hidden_units = encoder_hidden_units * 2

len(diff):  175
tmp len:  175
before:  711
fin:  886


In [6]:
x, m1, m2, _lens = next(DataGen)
for x_, m1_, m2_ in zip(x, m1, m2):
    print(DataGen.decode_sentence(x_))
    print(DataGen.decode_acts(m2_))
    print(DataGen.decode_slots(m1_))
    print("-----")

['and', 'what', 'is', 'the', 'price', 'ran']
['O', 'O', 'O', 'O', 'B-request', 'I-request']
['O', 'O', 'O', 'O', 'B-pricerange', 'I-pricerange']
-----
['i', 'need', 'expensive', 'kosher', 'restaurant']
['O', 'O', 'B-inform', 'B-inform', 'O']
['O', 'O', 'B-pricerange', 'B-food', 'O']
-----
['thank', 'you', 'can', 'i', 'have', 'the', 'adddress', 'phone', 'number']
['O', 'O', 'O', 'O', 'O', 'O', 'B-request', 'B-request', 'I-request']
['O', 'O', 'O', 'O', 'O', 'O', 'B-addr', 'B-phone', 'I-phone']
-----
['a', 'restaurant', 'south', 'part', 'of', 'town']
['O', 'O', 'B-inform', 'O', 'O', 'O']
['O', 'O', 'B-area', 'O', 'O', 'O']
-----
['i', 'need', 'a', 'expensive', 'restaurant']
['O', 'O', 'O', 'B-inform', 'O']
['O', 'O', 'O', 'B-pricerange', 'O']
-----
['an', 'expensive']
['O', 'B-inform']
['O', 'B-pricerange']
-----
['can', 'you', 'repeat', 'the', 'address', 'please']
['O', 'O', 'O', 'O', 'B-request', 'O']
['O', 'O', 'O', 'O', 'B-addr', 'O']
-----
['whats', 'its', 'number']
['O', 'O', 'B-re

In [7]:
slots_number = len(DataGen.slots_encode)
acts_number = len(DataGen.acts_encode)
print("slots_number: ", slots_number)
print("acts_number: ", acts_number)
print("vocab_size: ", DataGen.vocab_size)

slots_number:  26
acts_number:  18
vocab_size:  886


In [8]:
# MAKING EMBEDDING MATRIX

fs_model = fs.load_model("./model.bin")
embedd = np.zeros((vocab_size, input_embedding_size), dtype=np.float32)
for k, v in DataGen.vocab.items():
    embedd[v] = fs_model[k]
embedd[0] = [0]*100

In [9]:
tf.reset_default_graph()

lstm_size = 256
inputs = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask1 = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask2 = tf.placeholder(shape=(None, None), dtype=tf.int32)
lens = tf.placeholder(shape=(None,), dtype=tf.int32)

embeddings = tf.get_variable('embeddings', initializer=embedd, dtype=tf.float32, trainable=False)

encoded = tf.nn.embedding_lookup(embeddings, inputs)
cell = LSTMCell(lstm_size)

outputs, _ = tf.nn.dynamic_rnn(cell, encoded, sequence_length=lens, dtype=tf.float32)


seq_mask = tf.to_float(tf.greater(inputs, PAD))

# regularization:
#
# with tf.variable_scope('lm-reg'):
#     first_words = tf.slice(inputs, [0, 1], [-1, -1])
#     target = tf.pad(first_words, [[0, 0], [0, 1]])
#     nwp = layers.fully_connected(outputs, vocab_size, activation_fn=None)
#     lm_loss = seq2seq.sequence_loss(nwp, target, seq_mask)

with tf.variable_scope('mask-pred'):
    m1p = layers.fully_connected(outputs, slots_number, activation_fn=None)
    m2p = layers.fully_connected(outputs, acts_number, activation_fn=None)
    
    prediction_m1 = tf.argmax(tf.nn.softmax(m1p), axis = 2)
    prediction_m2 = tf.argmax(tf.nn.softmax(m2p), axis = 2)
    
    m1_loss = tf.reduce_sum(seq2seq.sequence_loss(m1p, mask1, seq_mask, average_across_batch=False, average_across_timesteps=False))
    m2_loss = tf.reduce_sum(seq2seq.sequence_loss(m2p, mask2, seq_mask, average_across_batch=False, average_across_timesteps=False))

# alternative loss:
#     log_likelihood_1, transition_params_1 = tf.contrib.crf.crf_log_likelihood(m1p, mask1, lens)
#     log_likelihood_2, transition_params_2 = tf.contrib.crf.crf_log_likelihood(m2p, mask2, lens)
    
lr = tf.Variable(0.0001, trainable=False)
opt = tf.train.GradientDescentOptimizer(lr)
train_op = opt.minimize(m1_loss + m2_loss)

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    avg_loss = -np.ones((3, ))
    alpha = 0.9
    
    pbar = tqdm(range(30000))
    for step in pbar:
        x, m1, m2, _lens = next(DataGen)
        res = sess.run([m1_loss, m2_loss, train_op], {inputs: x, mask1: m1, mask2: m2, lens: _lens})
        pbar.set_description(str(res[0]) + ' ' + str(res[1]))
        
#         print(res[0])
#         res = sess.run([lm_loss, m1_loss, m2_loss, train_op], {inputs: x, mask1: m1, mask2: m2, lens: _lens})
#         res = np.array(res[:-1])
        
#         if avg_loss[0] < 0:
#             avg_loss = res
        
#         avg_loss = alpha * avg_loss + (1-alpha) * res
#         pbar.set_description(' '.join('{:.2f}'.format(_) for _ in avg_loss))
        
        
        if(step%1000 == 0):
            
            x_dev, m1_dev, m2_dev, _lens_dev = next(DataGenDev)
            print("step: ", step)
            
            N = 3 # how many examples show to test:
            loss_cv_1, loss_cv_2, _, pred_1, pred_2 = sess.run([m1_loss, m2_loss, train_op, prediction_m1, prediction_m2], 
                                                    {inputs: x_dev, 
                                                      mask1: m1_dev,
                                                      mask2: m2_dev,
                                                      lens: _lens_dev})
            print("CV_loss_slots_1: ", loss_cv_1)
            print("CV_loss_acts_2: ", loss_cv_2)
#             print("CV_loss_acts: ", m2_cv_loss)
            
            for x_, m1_, m2_, p1, p2 in zip(x_dev[:N], m1_dev[:N], m2_dev[:N], pred_1[:N], pred_2[:N]):

                print("INPUT: ", DataGen.decode_sentence(x_))
                print("m2_tar: ", DataGen.decode_acts(m2_))
                print("m2_pred: ", DataGen.decode_acts(p2))                
                
                print("-----")
                
                print("m1_tar: ", DataGen.decode_slots(m1_))
                print("m1_pred: ", DataGen.decode_slots(p1))
                
                print("==========================")
    saver.save(sess=sess, save_path='nlu_model',global_step=30000)

663.629 593.854:   0%|          | 1/30000 [00:00<1:35:46,  5.22it/s]

step:  0
CV_loss_slots_1:  766.593
CV_loss_acts_2:  684.761
INPUT:  ['what', 'serves', 'is', 'in', 'the', 'north', 'part', 'of', 'town']
m2_tar:  ['O', 'B-request', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O']
m2_pred:  ['B-request', 'B-request', 'I-reqalts', 'B-request', 'B-request', 'B-request', 'B-request', 'B-request', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'B-food', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O']
m1_pred:  ['B-bye', 'B-addr', 'I-area', 'I-area', 'I-area', 'I-area', 'I-area', 'I-area', 'I-area', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['id', 'like', 'to', 'find', 'an', 'expensive', 'restaurant']
m2_tar:  ['O', 'O', 'O', 'O', 'O', 'B-inform', 'O']
m2_pred:  ['B-request', 'B-request', 'B-request', 'B-request', 'B-request', 'B-request', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'O', 'B-pricerange', 'O']
m1_pred:  ['O', 'B-addr', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

196.904 167.784:   3%|▎         | 1006/30000 [00:24<13:45, 35.11it/s]

step:  1000
CV_loss_slots_1:  191.995
CV_loss_acts_2:  142.0
INPUT:  ['whats', 'the', 'addre']
m2_tar:  ['O', 'O', 'B-request']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'B-addr']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['how', 'about', 'one', 'that', 'serves', 'malaysian', 'food']
m2_tar:  ['O', 'O', 'O', 'O', 'O', 'B-inform', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'O', 'B-food', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['what', 'is', 'its', 'addre', 'telephone', 'and', 'part', 'of', 'town']
m2_tar:  ['O', 'O', 'O', 'B-request', 'B-request', 'O', 'B-request', 'I-request', 'I-request']
m2_pred:  ['O', 'O', 'O', 'O', 'B-request', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'B-a

135.821 101.326:   7%|▋         | 2006/30000 [00:49<12:05, 38.56it/s]

...

34.6688 33.8869:  93%|█████████▎| 28006/30000 [11:48<00:50, 39.81it/s]

step:  28000
CV_loss_slots_1:  64.6647
CV_loss_acts_2:  38.615
INPUT:  ['type', 'of', 'serves']
m2_tar:  ['O', 'O', 'B-request']
m2_pred:  ['O', 'O', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'B-food']
m1_pred:  ['O', 'O', 'B-food', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['i', 'am', 'looking', 'for', 'creative', 'type', 'food']
m2_tar:  ['O', 'O', 'O', 'O', 'B-inform', 'O', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-food', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['and', 'for', 'good', 'luck', 'what', 'is', 'the', 'postal', 'code']
m2_tar:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-request', 'I-request']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-request', 'I-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O

26.4837 21.8155:  97%|█████████▋| 29008/30000 [12:12<00:25, 39.67it/s]

step:  29000
CV_loss_slots_1:  96.775
CV_loss_acts_2:  48.5396
INPUT:  ['a', 'restaurant', 'in', 'the', 'centre', 'part', 'of', 'town', 'serving', 'brazilian', 'food']
m2_tar:  ['O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'O', 'B-inform', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O', 'O', 'B-food', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O', 'O', 'B-food', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['what', 'is', 'the', 'number']
m2_tar:  ['O', 'O', 'O', 'B-request']
m2_pred:  ['O', 'O', 'O', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'B-phone']
m1_pred:  ['O', 'O', 'O', 'B-phone', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['any', 'kind']
m2_tar:  ['O', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O']
m1_pred:  ['O'

37.1094 30.076: 100%|██████████| 30000/30000 [12:36<00:00, 39.64it/s] 


In [11]:
file_writer = tf.summary.FileWriter('../logdir/', sess.graph)

## Это чисто черновик, где происходит какая-то фигня с bi-LSTM

In [ ]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

encoder_targets = tf.placeholder(shape=(None), dtype=tf.int32, name='encoder_targets')
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [ ]:
encoder_cell_b = LSTMCell(encoder_hidden_units)
encoder_cell_f = LSTMCell(encoder_hidden_units, reuse=None)

In [ ]:
# encoder_cell = LSTMCell(encoder_hidden_units, reuse=tf.get_variable_scope().reuse)
# encoder_cell_f = LSTMCell(encoder_hidden_units)
((encoder_fw_outputs,
encoder_bw_outputs),
(encoder_fw_final_state,
encoder_bw_final_state)) = (tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell_f,
                                cell_bw=encoder_cell_b,
                                inputs=encoder_inputs_embedded,
                                sequence_length=encoder_inputs_length,
                                dtype=tf.float32, time_major=TimeMajor))
    

In [ ]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h)

In [ ]:
from tensorflow.contrib import layers
next_word_pred = layers.fully_connected(encoder_outputs, vocab_size, ...)



mask1_pred = layers.fully_connected(encoder_outputs, mask1_vocab, ...)
mask2_pred = layers.fully_connected(encoder_outputs, mask2_vocab, ...)


In [ ]:
encoder_final_state_h

In [ ]:
W = tf.Variable(tf.random_uniform([encoder_hidden_units*2, vocab_size], -1, 1), dtype=tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [ ]:
logits = tf.add(tf.matmul(encoder_final_state_h, W), b)
# encoder_logits = tf.reshape(encoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [ ]:
logits

In [ ]:
encoder_targets

In [ ]:
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = encoder_targets)